In [ ]:
from __future__ import division, print_function

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, Conv2DTranspose
from tensorflow.keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D, UpSampling2D, Conv2D, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

class GAN():
    def __init__(self):
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3   #imagens passadas em grayscale com apenas um canal
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100   #dimensao do espaco latente (ruido)
        
        optimizer = Adam(0.0002, 0.5)   # taxa de aprendizado e beta1 do otimizador Adam
        
        # Criando e compilando o discriminador
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss = ['binary_crossentropy'], 
                                  optimizer = optimizer,
                                  metrics = ['accuracy'])
        
        # Construindo e compilando o gerador
        self.generator = self.build_generator()
        
        # Gerador recebe ruido na entrada e gera imagens
        z = Input(shape=(self.latent_dim))
        img = self.generator(z)
        
        # Somente o gerador e treinado no modelo da GAN
        self.discriminator.trainable = False
        
        # O discriminador recebe imagens geradas e determina sua validade
        validity = self.discriminator(img)
        
        # O modelo combinado treina o gerador com base na resposta do discriminador
        self.combined = Model(z, validity)
        self.combined.compile(loss = 'binary_crossentropy',
                             optimizer = optimizer)
        
        
    def build_generator(self):
        
        model = Sequential()
        
        model.add(Dense(384*4*4, activation='relu', input_dim=self.latent_dim))
        model.add(Reshape((4,4,384)))
        model.add(BatchNormalization())
        #model.add(UpSampling2D())
        model.add(Conv2DTranspose(192, kernel_size=5, strides=2, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        #model.add(UpSampling2D())
        model.add(Conv2DTranspose(96, kernel_size=5, strides=2, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Conv2DTranspose(self.channels, kernel_size=5, strides=2, padding='same'))
        model.add(Activation('tanh'))
        
        model.summary()
        
        noise = Input(shape=(self.latent_dim,))
        img = model(noise)
        
        return Model(noise, img)
    
    
    def build_discriminator(self):
        
        model = Sequential()
        
        model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=self.img_shape, padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(BatchNormalization())
        model.add(Conv2D(32, kernel_size=3, strides=1, padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(BatchNormalization())
        model.add(Conv2D(64, kernel_size=3, strides=2, padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(BatchNormalization())
        model.add(Conv2D(128, kernel_size=3, strides=1, padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(BatchNormalization())
        model.add(Conv2D(256, kernel_size=3, strides=2, padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        model.add(BatchNormalization())
        model.add(Conv2D(512, kernel_size=3, strides=2, padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        
        model.add(Flatten())
        model.summary()
                
        img = Input(shape=self.img_shape)
        
        features = model(img)
        validity = Dense(1, activation='sigmoid')(features)
        
        return Model(img, validity)
    
    
    def train(self, epochs, batch_size=128, sample_interval=50):
        
        (X_train, _),(_, _) = cifar10.load_data()
        
        # Reescalando dados de [0 255] (greyscale) para [-1 1]
        X_train = (X_train - 127.5)/127.5
        
        # Labels para falsos e verdadeiros
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        D_loss = []
        G_loss = []
        Acc = []
        
        for epoch in range(epochs):
            
            # Treinamento do Discriminador
            
            # Seleciona um conjunto aleatorio de imagens
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]
            
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            
            # Gera um conjunto de novas imagens
            gen_imgs = self.generator.predict(noise)
            
            # Treina o discriminador
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5*np.add(d_loss_fake, d_loss_real)
            
            # Treina Gerador
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            
            # Treina Gerador (usa indices como se dados criados fossem verdadeiros para induzir erro no discriminador)
            g_loss = self.combined.train_on_batch(noise, valid)
            
            # Plot do progresso do treinamento
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            
            D_loss.append(d_loss[0])
            G_loss.append(g_loss)
            Acc.append(d_loss[1])
            
            # Salvando imagens geradas com certo intervalo de epocas
            if epoch % sample_interval == 0:
                self.sample_images(epoch)
                self.save_model()
                
        return D_loss, G_loss, Acc

            
    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r*c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        
        # Reescalando imagens para [0 1]
        gen_imgs = 0.5*gen_imgs + 0.5
        
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt,:,:,:])
                axs[i,j].axis('off')
                cnt+=1
        fig.savefig("images/%d.png" % epoch)
        plt.close()
        
        
    def save_model(self):

        def save(model, model_name):
            model_path = "saved_model/%s.json" % model_name
            weights_path = "saved_model/%s_weights.hdf5" % model_name
            options = {"file_arch": model_path,
                        "file_weight": weights_path}
            json_string = model.to_json()
            open(options['file_arch'], 'w').write(json_string)
            model.save_weights(options['file_weight'])

        save(self.generator, "generator")
        save(self.discriminator, "discriminator")
        
if __name__ == '__main__':
    gan = GAN()
    D_loss, G_loss, Acc = gan.train(epochs=30000, batch_size=100, sample_interval=200)

W0913 10:36:43.365625  5876 deprecation.py:506] From c:\users\pedro_000\anaconda3\envs\deep_learning_1\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 16, 16, 16)        448       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 16, 16, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 16)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 16, 16, 16)        64        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 32)        4640      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0

W0913 10:36:45.086786  5876 deprecation.py:323] From c:\users\pedro_000\anaconda3\envs\deep_learning_1\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 6144)              620544    
_________________________________________________________________
reshape (Reshape)            (None, 4, 4, 384)         0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 4, 4, 384)         1536      
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 8, 8, 192)         1843392   
_________________________________________________________________
activation (Activation)      (None, 8, 8, 192)         0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 8, 8, 192)         768       
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 16, 16, 96)       

W0913 10:36:56.240320  5876 training.py:2197] Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
W0913 10:37:02.112732  5876 training.py:2197] Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?


...

6593 [D loss: 0.642245, acc.: 61.50%] [G loss: 0.754662]
6594 [D loss: 0.676253, acc.: 56.00%] [G loss: 0.809441]
6595 [D loss: 0.699431, acc.: 56.00%] [G loss: 0.812338]
6596 [D loss: 0.688317, acc.: 51.50%] [G loss: 0.809850]
6597 [D loss: 0.650545, acc.: 66.00%] [G loss: 0.837084]
6598 [D loss: 0.683267, acc.: 58.00%] [G loss: 0.815367]
6599 [D loss: 0.662391, acc.: 58.00%] [G loss: 0.832370]
6600 [D loss: 0.680967, acc.: 58.50%] [G loss: 0.838898]
6601 [D loss: 0.643825, acc.: 59.50%] [G loss: 0.878927]
6602 [D loss: 0.684051, acc.: 60.00%] [G loss: 0.814032]
6603 [D loss: 0.724362, acc.: 47.50%] [G loss: 0.757109]
6604 [D loss: 0.699067, acc.: 50.50%] [G loss: 0.769035]
6605 [D loss: 0.658229, acc.: 60.50%] [G loss: 0.855438]
6606 [D loss: 0.665068, acc.: 60.00%] [G loss: 0.854100]
6607 [D loss: 0.676785, acc.: 59.00%] [G loss: 0.823668]
6608 [D loss: 0.730067, acc.: 41.00%] [G loss: 0.758720]
6609 [D loss: 0.674473, acc.: 62.00%] [G loss: 0.986110]
6610 [D loss: 0.671556, acc.: 6

6737 [D loss: 0.673692, acc.: 61.50%] [G loss: 0.849474]
6738 [D loss: 0.710453, acc.: 48.50%] [G loss: 0.759111]
6739 [D loss: 0.697555, acc.: 51.50%] [G loss: 0.875055]
6740 [D loss: 0.665939, acc.: 59.50%] [G loss: 0.839608]
6741 [D loss: 0.709622, acc.: 47.50%] [G loss: 0.810920]
6742 [D loss: 0.678456, acc.: 57.50%] [G loss: 0.869799]
6743 [D loss: 0.696838, acc.: 53.00%] [G loss: 0.793619]
6744 [D loss: 0.685879, acc.: 54.00%] [G loss: 0.791760]
6745 [D loss: 0.679401, acc.: 59.50%] [G loss: 0.780946]
6746 [D loss: 0.676449, acc.: 58.00%] [G loss: 0.801323]
6747 [D loss: 0.668326, acc.: 59.50%] [G loss: 0.827703]
6748 [D loss: 0.714434, acc.: 48.50%] [G loss: 0.805890]
6749 [D loss: 0.684391, acc.: 60.50%] [G loss: 0.798619]
6750 [D loss: 0.642116, acc.: 65.50%] [G loss: 0.822956]
6751 [D loss: 0.675850, acc.: 64.00%] [G loss: 0.822024]
6752 [D loss: 0.706421, acc.: 47.00%] [G loss: 0.817527]
6753 [D loss: 0.696639, acc.: 50.00%] [G loss: 0.766333]
6754 [D loss: 0.657234, acc.: 6

6881 [D loss: 0.676503, acc.: 57.50%] [G loss: 0.811484]
6882 [D loss: 0.683507, acc.: 57.00%] [G loss: 0.740665]
6883 [D loss: 0.678706, acc.: 61.00%] [G loss: 0.762092]
6884 [D loss: 0.663083, acc.: 56.00%] [G loss: 0.747256]
6885 [D loss: 0.655102, acc.: 63.00%] [G loss: 0.816141]
6886 [D loss: 0.694962, acc.: 56.00%] [G loss: 0.739661]
6887 [D loss: 0.684538, acc.: 53.50%] [G loss: 0.785077]
6888 [D loss: 0.651997, acc.: 65.50%] [G loss: 0.834941]
6889 [D loss: 0.714778, acc.: 49.50%] [G loss: 0.793231]
6890 [D loss: 0.666491, acc.: 56.50%] [G loss: 0.907384]
6891 [D loss: 0.687390, acc.: 55.00%] [G loss: 0.794139]
6892 [D loss: 0.712816, acc.: 46.50%] [G loss: 0.810530]
6893 [D loss: 0.670849, acc.: 57.50%] [G loss: 0.823888]
6894 [D loss: 0.716989, acc.: 48.00%] [G loss: 0.854729]
6895 [D loss: 0.688161, acc.: 55.50%] [G loss: 0.871380]
6896 [D loss: 0.681890, acc.: 58.50%] [G loss: 0.789265]
6897 [D loss: 0.674434, acc.: 62.00%] [G loss: 0.757600]
6898 [D loss: 0.684927, acc.: 5

7025 [D loss: 0.717769, acc.: 51.50%] [G loss: 0.742787]
7026 [D loss: 0.717942, acc.: 46.00%] [G loss: 0.775321]
7027 [D loss: 0.683844, acc.: 54.00%] [G loss: 0.762455]
7028 [D loss: 0.691767, acc.: 51.50%] [G loss: 0.744287]
7029 [D loss: 0.667712, acc.: 56.00%] [G loss: 0.795814]
7030 [D loss: 0.665332, acc.: 59.00%] [G loss: 0.783250]
7031 [D loss: 0.669753, acc.: 56.50%] [G loss: 0.806013]
7032 [D loss: 0.694496, acc.: 50.50%] [G loss: 0.757428]
7033 [D loss: 0.678374, acc.: 58.00%] [G loss: 0.805815]
7034 [D loss: 0.688850, acc.: 54.50%] [G loss: 0.715037]
7035 [D loss: 0.671459, acc.: 59.00%] [G loss: 0.784165]
7036 [D loss: 0.678949, acc.: 55.00%] [G loss: 0.756258]
7037 [D loss: 0.668669, acc.: 56.00%] [G loss: 0.803553]
7038 [D loss: 0.660869, acc.: 57.50%] [G loss: 0.817581]
7039 [D loss: 0.679926, acc.: 52.00%] [G loss: 0.736066]
7040 [D loss: 0.666148, acc.: 59.50%] [G loss: 0.780792]
7041 [D loss: 0.664702, acc.: 61.50%] [G loss: 0.778885]
7042 [D loss: 0.677970, acc.: 5

7169 [D loss: 0.691846, acc.: 59.00%] [G loss: 0.776217]
7170 [D loss: 0.658515, acc.: 59.50%] [G loss: 0.764225]
7171 [D loss: 0.650797, acc.: 63.00%] [G loss: 0.776560]
7172 [D loss: 0.656938, acc.: 60.00%] [G loss: 0.756208]
7173 [D loss: 0.677929, acc.: 57.50%] [G loss: 0.814462]
7174 [D loss: 0.651902, acc.: 65.00%] [G loss: 0.826649]
7175 [D loss: 0.689977, acc.: 55.50%] [G loss: 0.777791]
7176 [D loss: 0.699955, acc.: 50.00%] [G loss: 0.807429]
7177 [D loss: 0.684179, acc.: 53.50%] [G loss: 0.767443]
7178 [D loss: 0.675646, acc.: 56.00%] [G loss: 0.801185]
7179 [D loss: 0.652047, acc.: 65.50%] [G loss: 0.784573]
7180 [D loss: 0.690203, acc.: 57.00%] [G loss: 0.703781]
7181 [D loss: 0.684567, acc.: 52.00%] [G loss: 0.789992]
7182 [D loss: 0.686378, acc.: 58.50%] [G loss: 0.810788]
7183 [D loss: 0.668900, acc.: 58.00%] [G loss: 0.725773]
7184 [D loss: 0.675114, acc.: 59.50%] [G loss: 0.717495]
7185 [D loss: 0.668747, acc.: 62.50%] [G loss: 0.755182]
7186 [D loss: 0.675038, acc.: 5